In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd

In [3]:
# from imp import reload

In [4]:
from utilities.data.get_data import get_data

In [5]:
from cbir.cbir import CBIRPredictor

In [6]:
# import cbir.cbir as cbir

In [7]:
# reload(cbir.CBIRPredictor)

In [8]:
query_params = {
    'START_AT': '2020-02-15',
    'END_AT': '2020-02-15',
}

In [9]:
df = get_data(query_params)

In [10]:
df.head()

,id,user_id,route_id,route_unmatched,model_version,path,created_at,descriptors,db_id,db_route_id,db_created_at,db_path,db_descriptors
0,1,1,1,False,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+00:00,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01...",37,10,2020-02-10 12:00:00+00:00,s3://climbicus-dev/route_images/dummy_data/gym...,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01..."
1,1,1,1,False,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+00:00,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01...",38,10,2020-02-10 12:00:00+00:00,s3://climbicus-dev/route_images/dummy_data/gym...,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01..."
2,1,1,1,False,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+00:00,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01...",39,10,2020-02-10 12:00:00+00:00,s3://climbicus-dev/route_images/dummy_data/gym...,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01..."
3,1,1,1,False,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+00:00,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01...",40,10,2020-02-10 12:00:00+00:00,s3://climbicus-dev/route_images/dummy_data/gym...,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01..."
4,1,1,1,False,dummy_version,s3://climbicus-dev/route_images/dummy_data/gym...,2020-02-15 09:00:00+00:00,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01...",41,11,2020-02-10 12:00:00+00:00,s3://climbicus-dev/route_images/dummy_data/gym...,"[b'\x93', b'N', b'U', b'M', b'P', b'Y', b'\x01..."


In [11]:
def reproduce(df):
    df = df.sample(10)
    cbir_predictor = CBIRPredictor()
    cbir_predictor.init_matcher("flann")
    returned_series = df.apply(lambda x: cbir_predictor.match_images(x['descriptors'], x['db_descriptors']), axis=1)
    df[['matches', 'dist']] = pd.DataFrame(returned_series.tolist(), index=returned_series.index)  
    return df

In [12]:
df = reproduce(df)

In [13]:
df.groupby('id').dist.nsmallest(2)

id      
2   128     224.0
9   679     219.0
10  720     182.0
17  1291    169.0
18  1400    198.0
21  1595    194.0
    1634    205.0
27  2080    205.0
36  2825    223.0
Name: dist, dtype: float64

There will be images that have no route id _ route_unmatched - incorrect. See which ones it matched with. 

Ignore where no route id and route_unmatched False.

With route id - correct.

Need to match each row. Then group by id, and get the lowest distances. 
Then group by status - route id / route_unmatched. 

In [ ]:
def get_stats(df):